# 整理歌单数据

使用油猴插件去除web端歌单只显示20首歌的限制，手动使用xpath提取出song_id
$x("//a[contains(@href,'song?id=')]/@href")
存入文件song_id.txt

In [1]:
from pathlib import Path
import re

text = Path('song_id.txt').read_text()

#正则提取所有数字串，组成列表
song_id = set(re.findall('\d+', text))

## 爬取歌词并保存

In [2]:
import requests
import json

def get_lyric(song_id):
    # 请求歌词报文
    url = f'http://music.163.com/api/song/lyric?id={song_id}&tv=1'
    res = requests.get(url).text
    obj = json.loads(res)
    
    # 清洗歌词
    if 'tlyric' in obj.keys():
        lyric = obj['tlyric']['lyric']

        # 去掉[]及内其内容
        lyric = re.sub('\[.*\]', '', lyric)

        # 中文间空格替换为回车
        # lyric = re.sub('[\u4e00-\u9fa5]( )[\u4e00-\u9fa5]', '\n', lyric)

        # 去除空行
        while '\n\n' in lyric:
            lyric = lyric.replace('\n\n', '\n')

        return lyric

    else:
        return None

In [5]:
for i, each in enumerate(song_id):
    print(f'{i+1} / {len(song_id)}', end='\r')
    lyric = get_lyric(each)
    if lyric:
        Path(f'./lyrics/{each}.txt').write_text(lyric)

## 整理成训练语料需要的格式

json

In [6]:
p = Path('./lyrics')

# 将歌词存为列表
data = []
for each in p.iterdir():
    text = each.read_text()

    # 中文间空行替换为逗号
    text = re.sub('[\u4e00-\u9fa5](\n)[\u4e00-\u9fa5]', '，', text)

    data.append(text)

# 保存列表到文件
json.dump(data, Path('train.json').open('w'), ensure_ascii=False)